In [ ]:
# default_exp Web

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
#export
import os
import sys
import shutil
from jinja2 import Environment, FileSystemLoader, Template

# Web functions 

> API details.

In [4]:
#hide
from nbdev.showdoc import *

cPath = os.path.dirname(os.getcwd())
cPath
sys.path.insert(0,cPath)

In [24]:
#export
from refreshment.School import StudySystem
from refreshment.Program import Program, Subject, Record, Lesson
from refreshment.munge import makePriorsForSubject

In [6]:
#hide
guide = StudySystem.load("./data/programs.json")
for x in guide.programs:
    print(x.name)
    for y in x.subjects:
        print("\"" + x.name + "\",\"" + y.name + "\"")

Cathedral
"Cathedral","Reading"
"Cathedral","Music"
"Cathedral","Chapel"
"Cathedral","HealthAndExercise"
"Cathedral","Shedules"
"Cathedral","Math"
"Cathedral","Mindful"
"Cathedral","Specialist"
"Cathedral","EmotionalLearning"
"Cathedral","Writing"


In [7]:
#hide
cath = guide.programs[0]
foo = cath.subjects
sorted(foo, key=lambda x: x.name)[:2]

[Chapel; [No Robbers|1|1|NoRobbers.mp4,CSB Lower School Distance Chapel April 7th, 2020|2|2|CSB Lower School Distance Chapel April 7th, 2020.mp4,CSB Lower School Distance Chapel April 21st, 2020|3|3|CSB Lower School Distance Chapel April 21st, 2020.mp4,CSB Lower School Distance Chapel April 28th, 2020|4|4|CSB Lower School Distance Chapel April 28th, 2020.mp4,CSB Lower School Distance Chapel May 5th, 2020|5|5|CSB Lower School Distance Chapel May 5th, 2020.mp4],
 EmotionalLearning; [emotions|1|1|emotions.mp4,fillbucket|2|2|fillbucket.mp4,creative|3|3|creative.mp4,Kimochis Bug|4|4|KimochisBug.mp4,grattitude|5|5|grattitude.mp4,Grat.422|6|6|Grat.422.mp4,Grat.4.21|7|7|Grat.4.21.mp4,Sharing Picture|8|8|SharingPicture.mp4,Gratitude Tree|9|9|GratitudeTree.mp4]]

In [8]:
#hide
template = Template('Hello {{ name }}!')
template.render(name=cath.name)

'Hello Cathedral!'

In [35]:
#export

class Render:
    def __init__(self, program, dir="../../web",template = "../templates"):
        self.program = program
        self.outputdir = os.path.join(dir,program.name)
        self.template = template
        
        self.file_loader = FileSystemLoader(self.template)
        self.env = Environment(loader=self.file_loader)
        self.resources = "resources"

        
    def basePath(self):
        return self.outputdir
    
    def renderLesson(self,sub,lesson):
        makePriorsForSubject(sub)
        template = self.env.get_template("lesson.html")
        resDir = "../../.."
        
        output = template.render(item=sub,
                                 resDir=resDir,
                                 less=lesson,
                                 program = self.program,
                                 styleDir=os.path.join("..",self.resources))
        subDir = os.path.join(self.outputdir,sub.name)
        
        if not os.path.exists(subDir):
            os.mkdir(subDir)
        f = open(os.path.join(subDir,"vid_" +str(lesson.key) + ".html"), "w")
        f.write(output)
        
    def renderSubject(self,sub):
        makePriorsForSubject(sub)
        template = self.env.get_template("subject.html")
        videos = [x for x in sub.lessons if x.fileName.endswith(".mp4")] 
        resources = [x for x in sub.lessons if not x.fileName.endswith(".mp4")]
        for x in sub.lessons:
            self.renderLesson(sub,x)
        resDir = "../../.."
        #print(sub.sequences)
        output = template.render(item=sub,
                                 videos=videos,
                                 res=resources,
                                 resDir=resDir,
                                 program = self.program,
                                 styleDir=os.path.join("..",self.resources))
        subDir = os.path.join(self.outputdir,sub.name)
        
        if not os.path.exists(subDir):
            os.mkdir(subDir)
        f = open(os.path.join(subDir,"index.html"), "w")
        f.write(output)
        
    def renderSchool(self):
        
        foo = self.program.subjects
        seq = sorted(foo, key=lambda x: x.name)

        if not os.path.exists(self.outputdir):
            os.mkdir(self.outputdir)
            
        for x in seq:
            self.renderSubject(x)
            
        template = self.env.get_template("school.html")
        output = template.render(program=self.program,
                                 seq=seq,
                                 styleDir=os.path.join(".",self.resources))
        
        f = open(os.path.join(self.outputdir,"index.html"), "w")
        f.write(output)
        f.close()
        
        self.addResources()
        
    def addResources(self):
        resDir = os.path.join(self.outputdir,self.resources)
        if not os.path.exists(resDir):
            os.mkdir(resDir)
        startDir = os.path.join(".", self.resources)
        src_files = os.listdir(startDir)
        
        for file_name in src_files:
            full_file_name = os.path.join(startDir, file_name)
            if os.path.isfile(full_file_name):
                shutil.copy(full_file_name, os.path.join(resDir, file_name))
                
        


In [60]:
#hide
baseRender = Render(cath)
baseRender.renderSchool()
#style = "background-color: #ABAD9E;"